In [1]:
# define the rest call
def restcall(rows):
  import requests, uuid, pytz
  from datetime import datetime
  from azure.identity import ClientSecretCredential
  from azure.storage.filedatalake import DataLakeServiceClient
  
  # Service Principal
  adl_account_name = ""
  client_id = ""
  client_secret = ""
  tenant_id = ""
  
  # connect to Azure Date Lake
  credential = ClientSecretCredential(tenant_id, client_id, client_secret)
  service_client = DataLakeServiceClient(account_url="https://{}.dfs.core.windows.net".format(adl_account_name), credential=credential)  
  
  # Get date
  tz_LA = pytz.timezone('America/Los_Angeles') 
  date_today = datetime.now(tz_LA).strftime("%Y-%m-%d")

  # Create the directory and sub-directory if necessary 
  file_system_client = service_client.get_file_system_client(file_system="dataset")
  file_system_client.create_directory("rest")
  directory_client = file_system_client.get_directory_client("rest")
  directory_client.create_sub_directory(date_today)
  directory_client = directory_client.get_sub_directory_client(date_today)

  # create file for each partition
  file_client = directory_client.create_file("rest-{}.json".format(uuid.uuid4()))
  
  offset = 0  # content offset
  for r in rows:
    url = "https://jsonplaceholder.typicode.com/photos/{}".format(r["photoId"])
    response = requests.get(url)
    
    content = response.text
    content_length = len(content)
    file_client.append_data(data=content, offset=offset, length=content_length)
    offset += content_length
    file_client.flush_data(offset)
  
  # close the file
  file_client.close()


In [2]:
from pyspark.sql import Row

parameters = [{"photoId" : i} for i in range(1, 2601)]

df = spark.createDataFrame(Row(**x) for x in parameters).repartition(6)
print(df.rdd.getNumPartitions())

# run REST Call in each partition
df.foreachPartition(restcall)

6

In [3]:
for p in parameters:
  url = "https://jsonplaceholder.typicode.com/photos/{}".format(p["photoId"])
  response = requests.get(url)
  response.json()

In [4]:
import uuid, pytz
from datetime import datetime
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings

def init_adl(storage_account_name, client_id, client_secret, tenant_id):
    
    try:  
        global service_client

        credential = ClientSecretCredential(tenant_id, client_id, client_secret)

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=credential)
    
    except Exception as e:
        print(e)

adl_account_name = "adlsgen22019"
client_id = "bf5735b3-b9c8-49ee-843b-d74101bc1e27"
client_secret = "bkiTUO4lY]-3ny9HU7sRrlhZcRyo[Fq/"
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"

init_adl(adl_account_name, client_id, client_secret, tenant_id)

tz_LA = pytz.timezone('America/Los_Angeles') 
date_today = datetime.now(tz_LA).strftime("%Y-%m-%d")

file_system_client = service_client.get_file_system_client(file_system="dataset")
file_system_client.create_directory("rest")
directory_client = file_system_client.get_directory_client("rest")
directory_client.create_sub_directory(date_today)
directory_client = directory_client.get_sub_directory_client(date_today)

file_client = directory_client.create_file("rest-{}.json".format(uuid.uuid4()))

offset = 0

parameters = [{"photoId" : i} for i in range(1, 2601)]
for p in parameters:
  import requests
  
  url = "https://jsonplaceholder.typicode.com/photos/{}".format(p["photoId"])
  response = requests.get(url)
  file_contents = response.text
  
  content_length = len(file_contents)
  file_client.append_data(data=file_contents, offset=offset, length=content_length)
  offset += content_length
  file_client.flush_data(offset)

file_client.close()


In [5]:
from datetime import datetime
import pytz

now = datetime.now()
print("now =", now)

dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)	

now = datetime.now()
timestamp = datetime.timestamp(now)
print("timestamp =", timestamp)

tz_LA = pytz.timezone('America/Los_Angeles') 
datetime_LA = datetime.now(tz_LA)
print("LA time:", datetime_LA.strftime("%Y-%m-%d %H:%M:%S"))